<a href="https://colab.research.google.com/github/tsanzxc456/NLP/blob/master/lab5_Team_Yellow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter
import json
import requests
!mkdir -p majority_labels
!mkdir -p Naive_Bayes

## Fetch the data

In [2]:
data = pd.read_csv("https://nctu_team_yellow.imfast.io/nlp_final_project/train_gold.csv", delimiter=',', header=None, names=['idx', 'text', 'reply', 'categories', 'mp4'])
dev_data = pd.read_csv("https://nctu_team_yellow.imfast.io/nlp_final_project/dev_unlabeled.csv", delimiter=',', header=None, names=['idx', 'text', 'reply'])[1:]
train_data, test_data = train_test_split(data, test_size=0.1, random_state=43)

train_data.sample(5)

,idx,text,reply,categories,mp4
26639,26639,I used to be a psycho ass bitch when I was wit...,NaN,"[""idk""]",ef42a8cbd387818768d89a9806fdfd97.mp4
11835,11835,Now that we know who the essential workers are...,NaN,"[""applause""]",9b63f04e19f73f724d688eea1e62ca14.mp4
31045,31045,.@OANN Poll “Gives President Trump a 52% Appro...,NaN,"[""no"",""seriously"",""facepalm"",""omg""]",1bc0c284efc0248643651404ff6236eb.mp4
16016,16016,Do you think Fox News is a danger to the USA? ...,"Yes, definitely","[""agree"",""yes""]",0eacf9770d45e333f6156124267f8010.mp4
22558,22558,I don’t like when someone screenshots my SC,NaN,"[""ok""]",23e4ad1b7c3781b340cf3be84a043475.mp4


## Number of samples that have N categories, for 1 <= N <= 6

In [3]:
categories = ["agree", "applause", "awww", "dance", "deal_with_it", "do_not_want", "eww", "eye_roll", "facepalm", "fist_bump", "good_luck", "happy_dance", "hearts", "high_five", "hug", "idk", "kiss", "mic_drop", "no", "oh_snap", "ok", "omg", "oops", "please", "popcorn", "scared", "seriously", "shocked", "shrug", "sigh", "slow_clap", "smh", "sorry", "thank_you", "thumbs_down", "thumbs_up", "want", "win", "wink", "yawn", "yes", "yolo", "you_got_this"]
N_categories_list = [0,0,0,0,0,0]
train_cate_list = train_data.categories.tolist()

for k in range(len(train_cate_list)):
  x = 0
  for i in range(0,len(categories)):
    if train_cate_list[k].find(categories[i]) != -1:
      x = x + 1
  N_categories_list[x-1] = N_categories_list[x-1] + 1

print("Number of samples that have 1 categorie :", N_categories_list[0])
print("Number of samples that have 2 categories :", N_categories_list[1])
print("Number of samples that have 3 categories :", N_categories_list[2])
print("Number of samples that have 4 categories :", N_categories_list[3])
print("Number of samples that have 5 categories :", N_categories_list[4])
print("Number of samples that have 6 categories :", N_categories_list[5])


Number of samples that have 1 categorie : 20180
Number of samples that have 2 categories : 5316
Number of samples that have 3 categories : 1975
Number of samples that have 4 categories : 753
Number of samples that have 5 categories : 348
Number of samples that have 6 categories : 228


## Category distribution (how many times does each category appear)

In [4]:
categories_distrubution = {}
for cate in categories:
  categories_distrubution[cate] = 0

for k in range(len(train_cate_list)):
  for i in range(0,len(categories)):
    if train_cate_list[k].find(categories[i]) != -1:
      categories_distrubution[categories[i]] = categories_distrubution[categories[i]] + 1

categories_distrubution

{'agree': 2678,
 'applause': 3091,
 'awww': 887,
 'dance': 1207,
 'deal_with_it': 386,
 'do_not_want': 301,
 'eww': 826,
 'eye_roll': 1098,
 'facepalm': 1795,
 'fist_bump': 267,
 'good_luck': 640,
 'happy_dance': 714,
 'hearts': 305,
 'high_five': 489,
 'hug': 2925,
 'idk': 970,
 'kiss': 165,
 'mic_drop': 154,
 'no': 2125,
 'oh_snap': 800,
 'ok': 784,
 'omg': 1394,
 'oops': 735,
 'please': 548,
 'popcorn': 527,
 'scared': 751,
 'seriously': 1802,
 'shocked': 1519,
 'shrug': 768,
 'sigh': 1599,
 'slow_clap': 1215,
 'smh': 1698,
 'sorry': 304,
 'thank_you': 765,
 'thumbs_down': 196,
 'thumbs_up': 619,
 'want': 562,
 'win': 934,
 'wink': 517,
 'yawn': 414,
 'yes': 2459,
 'yolo': 182,
 'you_got_this': 742}

## 10 most common pairs of co-occurring categories

In [5]:
categories_co_occurring_list = []
for i in range(len(categories)):
  for cate in categories[i:]:
    if categories[i] != cate :
      categories_co_occurring_list.append(categories[i] + ',' + cate)

categories_co_occurring_dict = {}
for co_cate in categories_co_occurring_list:
  categories_co_occurring_dict[co_cate] = 0


for k in range(len(train_cate_list)):
  for i in range(len(categories)):
    for cate in categories[i:]:
      if train_cate_list[k].find(categories[i]) != -1 and train_cate_list[k].find(cate) != -1 and categories[i] != cate:
        categories_co_occurring_dict[categories[i]+','+cate] = categories_co_occurring_dict[categories[i]+','+cate] + 1

In [6]:
co_occur_counter = Counter(categories_co_occurring_dict)
co_occur_counter.most_common(10)

[('facepalm,seriously', 1063),
 ('applause,slow_clap', 1037),
 ('agree,yes', 995),
 ('sigh,smh', 972),
 ('facepalm,sigh', 951),
 ('facepalm,smh', 900),
 ('seriously,sigh', 798),
 ('dance,happy_dance', 714),
 ('seriously,smh', 706),
 ('omg,shocked', 698)]

# Part 2

## Majority Prediction (predict on the same 6 majority labels)

### Extracting 6 most popular labels.

In [7]:
categories_distrubution_counter = Counter(categories_distrubution)
majority_labels = categories_distrubution_counter.most_common(6)

In [8]:
majority_labels_list = [majority_label[0] for majority_label in majority_labels]
majority_labels_lists = [majority_labels_list for x in range(len(dev_data))]
dev_result = pd.read_csv("https://nctu_team_yellow.imfast.io/nlp_final_project/dev_unlabeled.csv", delimiter=',', header=None, names=['idx', 'text', 'reply'])[1:]
tmp_reply = dev_result.reply.tolist()
for (i,reply) in enumerate(tmp_reply):
  if type(reply) != str:
    tmp_reply[i] = ''
tmp_idx = dev_result.idx.tolist()
for (i,idx) in enumerate(tmp_idx):
  tmp_idx[i] = int(tmp_idx[i])
dev_result = dev_result[['text']]
dev_result.insert(0,"idx",tmp_idx),
dev_result.insert(1,"categories",majority_labels_lists)
dev_result.insert(2,"reply",tmp_reply)
dev_result.to_json('./majority_labels/dev.json',orient='records',force_ascii=False,lines=True)

## Naive Bayes


### Vectorizing data using default parameters


In [9]:
vectorizer = TfidfVectorizer() 
vectorizer.fit(train_data['text'])
train_feature = vectorizer.transform(train_data['text'])
test_feature = vectorizer.transform(test_data['text'])
dev_feature = vectorizer.transform(dev_data['text'])

train_feature.shape, test_feature.shape, dev_feature.shape

((28800, 33132), (3200, 33132), (4000, 33132))

### Extract first GIF category in each sample.

In [10]:
# The following is done for simplification
test_cate_list = test_data.categories.tolist()

train_label_first = [train_cate[2:min(train_cate.find('"]'),train_cate.find(',"'))-1] for train_cate in train_cate_list]
test_label_first = [test_cate[2:min(test_cate.find('"]'),test_cate.find(',"'))-1] for test_cate in test_cate_list]

### Train and Predict !!

In [11]:
model = MultinomialNB()
model.fit(train_feature.toarray(),train_label_first)
pred_label = model.predict(test_feature.toarray())

### Evaluation (for test data)

In [12]:
print(accuracy_score(test_label_first, pred_label))
print(classification_report(test_label_first, pred_label, digits=3))

0.1953125
              precision    recall  f1-score   support

       agree      0.140     0.544     0.223       298
    applause      0.205     0.149     0.172       235
        awww      0.000     0.000     0.000        82
       dance      0.000     0.000     0.000        80
deal_with_it      0.000     0.000     0.000        37
 do_not_want      0.000     0.000     0.000        10
         eww      0.214     0.043     0.072        69
    eye_roll      0.275     0.114     0.161       123
    facepalm      0.000     0.000     0.000        38
   fist_bump      0.000     0.000     0.000        21
   good_luck      0.000     0.000     0.000        68
 happy_dance      0.000     0.000     0.000        56
      hearts      0.000     0.000     0.000        32
   high_five      0.000     0.000     0.000        30
         hug      0.195     0.968     0.325       310
         idk      0.000     0.000     0.000        26
        kiss      0.000     0.000     0.000        25
    mic_drop     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### For each sample in eval file, pick 6 highest probability categories.

In [13]:
probabilities = model.predict_proba(dev_feature.toarray())
probabilities_list = probabilities.tolist()
pred_dev_label = []

for i in range(0,len(probabilities_list)):
  ranks = sorted( [(x,i) for (i,x) in enumerate(probabilities_list[i])], reverse=True )
  values = []
  cate_indexs = []
  predict_cates = []
  for x,i in ranks:
      if x not in values:
          values.append( x )
          cate_indexs.append( i )
          if len(values) == 6:
              break
  predict_cates = [categories[k] for k in cate_indexs]
  pred_dev_label.append(predict_cates)

In [14]:
dev_result2 = pd.read_csv("https://nctu_team_yellow.imfast.io/nlp_final_project/dev_unlabeled.csv", delimiter=',', header=None, names=['idx', 'text', 'reply'])[1:]
tmp_reply = dev_result2.reply.tolist()
for (i,reply) in enumerate(tmp_reply):
  if type(reply) != str:
    tmp_reply[i] = ''
tmp_idx = dev_result2.idx.tolist()
for (i,idx) in enumerate(tmp_idx):
  tmp_idx[i] = int(tmp_idx[i])
dev_result2 = dev_result2[['text']]
dev_result2.insert(0,"idx",tmp_idx),
dev_result2.insert(1,"categories",pred_dev_label)
dev_result2.insert(2,"reply",tmp_reply)
dev_result2.to_json('./Naive_Bayes/dev.json',orient='records',force_ascii=False,lines=True)
